In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load the sentiment-response dataset
df = pd.read_excel('result.xlsx')
sentiments = df['sentiment'].tolist()
responses = df['response'].tolist()

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(responses))

C:\Users\91932\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\91932\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91932\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To 

In [ ]:


# Preprocess the data using the tokenizer
inputs = tokenizer(sentiments, padding=True, truncation=True, max_length=512, return_tensors='pt')
labels = torch.tensor([responses.index(r) for r in responses])

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=32,
    learning_rate=5e-5,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=5,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False
)

# Define the trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=(inputs, labels)
)

# Fine-tune the language model
trainer.train()

# Save the fine-tuned model
trainer.save_model('./fine-tuned-bert')

# Generate email responses using the fine-tuned model
model = BertForSequenceClassification.from_pretrained('./fine-tuned-bert')
model.eval()

sentiment = "Positive"
input_ids = tokenizer.encode(sentiment, return_tensors='pt')
output = model(input_ids=input_ids)[0]
response_id = torch.argmax(output).item()
response = responses[response_id]

